## Preprocessing

### Estrazione frasi
Estraiamo un campione di frasi a partire dai file conllu di Wikipedia italiana elaborata. 

In [13]:
import os
import re
import pandas as pd
from tqdm import tqdm

In [14]:
def get_id(line):
    sent_id = re.sub(r'\D', '', line)
    return sent_id

def get_text(line):
    sent = line[9:].rstrip('\n')
    return sent

def get_sentences(file_conllu):
    sentences = []
    ids = []
    with tqdm(total=sum(1 for _ in open(file_conllu, 'r', encoding='utf-8')), desc=f'Progresso estrazione frasi del file {file_conllu}') as pbar:
        for line in open(file_conllu, 'r', encoding='utf-8'):
            pbar.update(1)  # Aggiorna la barra di avanzamento
            if line.startswith("# sent_id"):
                current_id = get_id(line)  
                ids.append(current_id)
            elif line.startswith("# text"):
                current_sent = get_text(line)
                sentences.append(current_sent)
    return ids, sentences

In [15]:
#nel caso in cui dovessimo itereare sui diversi file nella dir
"""ds_directory = "C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/wiki_conllu"
ds_files = []
for file_name in os.listdir(ds_directory):
    file_path = os.path.join(ds_directory, file_name)
    ds_files.append(file_path)  #ottengo il nome dei diversi file all'interno della directory
print(ds_files)"""

'ds_directory = "C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/wiki_conllu"\nds_files = []\nfor file_name in os.listdir(ds_directory):\n    file_path = os.path.join(ds_directory, file_name)\n    ds_files.append(file_path)  #ottengo il nome dei diversi file all\'interno della directory\nprint(ds_files)'

In [16]:
"""data_df = pd.DataFrame(columns=["id", "text"])
data_df["id"], data_df["text"] = get_sentences(ds_files[0])   #per ora lavoro su un singolo file"""

'data_df = pd.DataFrame(columns=["id", "text"])\ndata_df["id"], data_df["text"] = get_sentences(ds_files[0])   #per ora lavoro su un singolo file'

In [17]:
data_df = pd.read_csv("data/csv/sample_rs42.csv", encoding ="utf-8")

In [46]:
data_df.tail()

,sent_id,text,gulpease
999994,1147841,Il Kentucky può essere diviso in cinque region...,60
999995,546403,Wescott ha avuto cariche in diverse associazio...,34
999996,1373331,La cerchia di mura non racchiudeva nel medioev...,42
999997,572606,"I Memphis Grizzlies arrivarono quarti nella ""S...",50
999998,67405,La musica elettroacustica abbraccia l'insieme ...,35


In [19]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999999 entries, 0 to 999998
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   sent_id   999999 non-null  int64 
 1   text      999999 non-null  object
 2   gulpease  999999 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 22.9+ MB


In [61]:
INIZIO = 0
fine = 50
samplino = data_df.iloc[INIZIO:fine] 

In [62]:
samplino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sent_id   50 non-null     int64 
 1   text      50 non-null     object
 2   gulpease  50 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.3+ KB


In [63]:
data_df.iloc[fine-1]

sent_id                                                651531
text        Nei suoi, circa, dodici anni di regno (Sesto A...
gulpease                                                   39
Name: 49, dtype: object

In [64]:
samplino.iloc[len(samplino)-1]

sent_id                                                651531
text        Nei suoi, circa, dodici anni di regno (Sesto A...
gulpease                                                   39
Name: 49, dtype: object

### Estrazione degli indici di leggibilità per ogni frase

Utilizziamo l'API _read-it_ per calcolare gli indici di complessità di ogni frase.

In [26]:
import requests
import sys
import json

In [65]:

SERVER_PATH = "http://api.italianlp.it"
#SERVER_PATH = "http://itanlp-gpu.ilc.cnr.it:13000"

def load_document(text, i, x):
    try:
        r = requests.post(SERVER_PATH + '/documents/',           # carica il documento nel database del server
                        data={'text': text,                    # durante il caricamento viene eseguita un'analisi linguistica necessaria per calcolare la leggibilità
                            'lang': 'IT',
                            'extra_tasks': ["readability"]     # chiede al server di calcolare anche la leggibilità del docuemnto
                    })
        r.raise_for_status()  
        doc_id = r.json()['id']                           # id del documento nel database del server, che serve per richiedere i risultati delle analisi     
        return doc_id
    except requests.RequestException as e:
        print(f"Errore nel caricamento del documento: {e} \n ---------- Documento: ----------\n{text}")
        with open('frasi_saltate_val.txt', 'w', encoding='utf-8') as frasi_saltate:
            frasi_saltate.write(f"Iterazione saltata: {i+INIZIO} - {(i+INIZIO)+x} \nDOCUMENTO SALTATO \n{text}")
        return None
    

def get_doc_score(doc_id, i, x):
    try:
        r = requests.get(SERVER_PATH + '/documents/details/%s' % doc_id)
        r.raise_for_status()
        result = r.json()
        return result
    except requests.RequestException as e:
        print(f"Errore nel recupero dei risultati: {e}\n Iterazione: {i+INIZIO} - {(i+INIZIO)+x}: ")
        return None




In [14]:
#facciamo una prova con un testo di poche frasi
#text = "Il cielo notturno era illuminato da milioni di stelle scintillanti. La vita è un viaggio, non una destinazione. La musica è il linguaggio universale dell'umanità. Le piccole cose nella vita spesso portano le più grandi gioie. Non esiste un ascensore per il successo; devi prendere le scale. Ogni giorno è una nuova opportunità per essere felici. I fiori non si preoccupano di come sbocceranno; semplicemente lo fanno. La risata è il miglior antidoto contro lo stress. La gentilezza è un linguaggio che i sordi possono sentire e i ciechi possono vedere. Nella semplicità risiede la vera bellezza."


#doc_id = load_document(text, None, None)
#r_score = get_doc_score(doc_id, None, None)

#vediamo com'è strutturato l'oggetto che viene restituito dall'API
#print(r_score)

In [95]:
#qui iteriamo sui risultati ottenuti per un singolo documento
def get_sen_scores(doc_scores):
    all_sent_rscore = []
    all_sent_text = []
    for sent_results in doc_scores['sentences']['data']:
        all_sent_rscore.append(sent_results['readability_score_all'])
        all_sent_text.append(sent_results['raw_text'])
    return all_sent_text, all_sent_rscore

#si passa all'api un doc di 1000 frasi per volta.
def readability_extraction(sen_list, x): #x è il numero di frasi che si vogliono calcolare per iterazione 
    for i in tqdm(range(0, len(sen_list), x), desc=f"Progresso nell'estrazione degli indici di complessità delle frasi (ogni iterazione corrisponde a {x} frasi)"):
        doc = "\n".join(sen_list[i:i+x])
        doc_id = load_document(doc, i, x)
        r_scores = get_doc_score(doc_id, i, x)
        with open(f"data/readit_scores/json_scores{i+INIZIO}-{(i+INIZIO)+x}.json", "w") as f:
            json.dump(r_scores, f)
            #if i + x < len(sen_list):
            #    f.write(',')  # Aggiungiamo una virgola se non siamo all'ultima iterazione
      
    #all_scores.append(r_scores)
    

#qui si ottiene la struttura dati finale dove per ogni frase abbiamo lo score ottenuto da read_it

'''
def get_final_data(result_dict):
    text_list = []
    score_list = []
    for doc_scores in result_dict:
        text, score = get_sen_scores(doc_scores)
        text_list = text_list + text
        score_list = score_list + score
    return text_list, score_list
'''


'\ndef get_final_data(result_dict):\n    text_list = []\n    score_list = []\n    for doc_scores in result_dict:\n        text, score = get_sen_scores(doc_scores)\n        text_list = text_list + text\n        score_list = score_list + score\n    return text_list, score_list\n'

In [66]:
all_results = readability_extraction(samplino["text"].tolist(), 50)

Progresso nell'estrazione degli indici di complessità delle frasi (ogni iterazione corrisponde a 50 frasi):   0%|          | 0/1 [00:00<?, ?it/s]

DOCUMENTO DA CARICARE: Il trio non era inizialmente interessato a lavorare nell'azienda di famiglia.
La ragazza riesce a convincere Lachlan a lasciar perdere promettendogli di copulare con lui.
In quanto campione del mondo uscente, egli avrebbe avuto comunque il diritto di partecipare al torneo dei candidati dell'edizione successiva, tuttavia decise di non avvalersi di tale diritto e di ritirarsi dalla competizione per il titolo mondiale, pur rimanendo attivo negli scacchi agonistici fino al 1970.
Scoperto nel 2000, presenta un'orbita caratterizzata da un semiasse maggiore pari a 2,7742634 UA e da un'eccentricità di 0,1087025, inclinata di 3,73822° rispetto all'eclittica.
Definizione dell'Autorità per l'Energia Elettrica e il Gas.
Il progetto del Vindicator nacque (con la sigla di progetto V-156) nel corso del 1935 quando il Bureau of Aeronautics (BuAer), dipartimento della United States Navy deputato alla gestione del materiale e tecnologia della propria componente aerea, avanzò alla 

Progresso nell'estrazione degli indici di complessità delle frasi (ogni iterazione corrisponde a 50 frasi): 100%|██████████| 1/1 [00:11<00:00, 11.40s/it]


In [93]:
#codice per estrarre score dai json
#iteriamo sui diversi file json nella directory
json_dir = "C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/readit_scores/training"
ds_files = []
for file_name in os.listdir(json_dir):
    file_path = os.path.join(json_dir, file_name)
    ds_files.append(file_path) #ottengo il nome dei diversi file all'interno della directory

In [96]:
#si leggono i file json per ottenere lista di frasi e lista di punteggi read-it
all_sent = []
all_scores = []
for file_name in tqdm(ds_files, desc="Processing files"):
    with open(file_name, "r", encoding="utf-8") as json_file:
        result = json.load(json_file)
        #print(result["sentences"]["count"])
    if result != None:
        doc_sen, doc_scores = get_sen_scores(result)
        all_sent = all_sent + doc_sen
        all_scores = all_scores + doc_scores
    else: 
        print(f'File vuoto: {file_name}\n')
        

Processing files: 100%|██████████| 1000/1000 [03:08<00:00,  5.29it/s]


### Creazione e valutazione del dataset finale

Si ottiene una struttura dataframe con _id_, _testo e _leggibilità_ per ogni frase. Dopodiché valutiamo le caratteristiche dei dati ottenuti

In [69]:
len(all_sent)

1000000

In [45]:
#si crea un dataframe con tutte le informazioni che ci interessano
data_set = pd.DataFrame(columns=["id", "text", "readability"])
data_set["text"] = all_sent
data_set["readability"] = all_scores
data_set["id"] = range(1, len(data_set) + 1)
data_set.head()

,id,text,readability
999995,999996,L'Ordine ammetteva in tutto 30 dame esclusa la...,74.445784
999996,999997,"""il suo cavallo gli era stato ucciso sotto la ...",87.114897
999997,999998,Oltre a un istituto comprensivo (scuola elemen...,15.050005
999998,999999,Riconobbe inoltre che le rocce potessero esser...,65.320529
999999,1000000,Nessuno di quegli aerei ritornò indietro e l'a...,26.933706


In [47]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   id           1000000 non-null  int64  
 1   text         1000000 non-null  object 
 2   readability  968304 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 22.9+ MB


Si controlla l'ordine dei dati. Da tenere in considerazione il fatto che "all_results" è in forma di lista contenente i dizionari dei risultati dell'API per ogni "documento" (ovvero ogni x frasi). Quindi l'indice utilizzato per gli elementi di "all_results" per controllare l'ordine sarà sempre compreso tra 0 e x. Mentre quello del nostro dataframe sarà l'equivalente + x*y (dove y, è il numero del documento che stiamo prendendo in considerazione)

In [55]:
# print(len(results_dict[0]))

36


In [56]:
# #ora controlliamo che l'ordine sia corretto...
# tupla = data_set[data_set["id"]==35]
# da_result = results_dict[0]['sentences']['data'][34] #primo "documento"
# print(f"---- Nel dataframe finale ----\n Testo: '{tupla['text'].item()}' - R_score: {tupla['readability'].item()}\n\n") 
# print(f"---- Nei risultati ottenuti dall'API ----\n Testo: '{da_result['raw_text']}' - R_score: {da_result['readability_score_all']}")

---- Nel dataframe finale ----
 Testo: 'Durante il regno di Ḥasan Kuçek, non tutti i Chupanidi restarono leali.' - R_score: 43.5424813612392


---- Nei risultati ottenuti dall'API ----
 Testo: 'la città non veniva servita da tre anni.' - R_score: 30.9012389143344


In [57]:
# tupla = data_set[data_set["id"]==49]
# da_result = results_dict[1]['sentences']['data'][48] #secondo "documento"
# print(f"---- Nel dataframe finale ---- \n Testo: {tupla['text'].item()} - R_score: {tupla['readability'].item()}\n\n") 
# print(f"---- Nei risultati ottenuti dall'API ---- \n  Testo: {da_result['raw_text']} - R_score: {da_result['readability_score_all']}")

---- Nel dataframe finale ---- 
 Testo: Il verdetto è un film del 1948, diretto dal regista Lewis Allen. - R_score: 39.4039916534288


---- Nei risultati ottenuti dall'API ---- 
  Testo: La nazionale di calcio della Polonia (pol. "Reprezentacja Polski w piłce nożnej mężczyzn") è la rappresentativa calcistica della Polonia ed è posta sotto l'egida della federazione calcistica polacca. - R_score: 58.4271426139914


In [33]:
valori_nulli = data_set[data_set["readability"].isnull()]
valori_nulli #vediamo che le frasi troppo corte non restituiscono un valore di leggibilità. 

,id,text,readability
19,20,Tra le sue caratteristiche:,NaN
40,41,Tratturo Magno).,NaN
59,60,È un portiere goleador.,NaN
69,70,Corte di appello di Catania,NaN
73,74,École Supérieure des Beaux-Arts,NaN
...,...,...,...
999863,999864,Europa League 2017-2018,NaN
999866,999867,Dompierre-sur-Helpe,NaN
999896,999897,Il naufragio del Forfashire.,NaN
999916,999917,Sono poliedri di Keplero:,NaN


In [91]:
#data_set = data_set.dropna()
data_set.to_csv("ds_leggibilita2.csv", index=False) #una volta pronto salviamo il file

#### Controlli 

In [137]:
len(data_df[data_df['text'].isin(data_set['text'])]) 

983587

In [141]:
data_set[~data_set['text'].isin(data_df['text'])]

,id,text,readability
39,40,Curiosamente la strada ripercorre a tratti le ...,36.695214
40,41,Tratturo Magno).,NaN
57,58,la sua produzione storica dalla dissoluzione d...,22.683105
58,59,"Secondo il ""Red Book"" del 2007 il paese ha ris...",94.245346
462,463,J.-H.,NaN
...,...,...,...
999581,999582,Roma conquistò l'intera Illiria nel 168 a.C.,46.001600
999847,999848,La città è governata dal Consiglio città di Su...,55.131287
999848,999849,Storia.,NaN
999949,999950,Il 27 gennaio 2007 su E!,0.989726


In [138]:
frasi_mancanti = data_df[~data_df['text'].isin(data_set['text'])]

In [139]:
len(frasi_mancanti)

16412